# Overview
For this project, we'll use a data set downloaded from [climate.gov](https://www.climate.gov/maps-data/dataset/daily-temperature-and-precipitation-reports-data-tables).  We'll focus on reshaping and filtering the data so we can use `numpy` to perform several analyses on the data.

This notebook guides you through performing the analysis.  While I encourage you to explore other methods of answering the analysis questions - I'd like you to follow my path through this notebook closely, as it exposes you to several important concepts when using `numpy`, as well as common data reshaping exercising that we'll see throughout the semester.

## Getting the data
This project is set up with DVC, so do a `dvc pull` to get the `temperature.csv` file.  This file contains daily temperature readings (max and min) from two weather reporting stations in NJ - Elizabeth NJ (Station USC00282644) and Conoe Brook NJ (Station USC00281335) - from 1893 until the end of April 2020.  We'll be using the data to track some statistics about daily temperatures in NJ over time.  For the purposes of this project, we don't care much about having two temperature samples for a given day - we just want one.  

## Analysis
This analysis should answer the following questions:
1. What is the hottest (max temp) day of the year, on average?
2. What is the coldest (min temp) day of the year, on average?
3. Which are the hottest 20 Septembers on record, based on calculating each year's average temperature for the onth of September?
4. When was there the longest heat wave in NJ in 2016?  To do this, we'll compute the average maximum temperature for each day of the year (0-365), calculate the difference between those averages and each day of 2016, and then find the maximum stretch of days that were +2 degrees over the yearly norm for that day.

# Setup and Utilities
This first cell sets up the imports we'll need.  `numpy` is  the most important, however I've also imported several utilities from the `datetime` module, which we'll be using to transform dates in a variety of places.  There is also a simple function to transform the station identifier (i.e. USC00281335) into a simple integer (the rightmost digit), so we can construct `numpy` arrays with station ids in it (remember, `numpy` arrays must contain the same data type throughout).

In [8]:
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

# A date object representing the date of the first observation found within temperature.csv.
# This object is used in day_transform an date_to_day.  Generally, throughout your analysis, 
# dates will be represented by an integer - the number of days since Jan 1 1893.  Again, this
# is primarily because numpy arrays need homogenous date - so we end up converting regular 
# date strings to integers for convenience.
begin = date(1893, 1, 1)

# d is the number of days (integer) since the beginning of the data (1/1/1893). This utility 
# function transforms d into an array. [0] is the year, [1] is the month, [2] is the day of the YEAR.
# Important - [2] is NOT the day of the month, it is the day since January 1st of the given year.
# This will be convenient in the analysis.
def day_transform(d):
    # d is days since begin
    _date = begin + timedelta(days=int(d))
    _by = date(_date.year, 1, 1)
    return [_date.year, _date.month, (_date - _by).days]

# d is a date string, initially as bytes.  We decode the byte string into UTF-8 string and construct
# A date object from it.  We then calculate the number of days since 1/1/1893 and return that as an 
# integer.
def date_to_day(d):
    s = d.decode("utf-8")
    date_obj = datetime.strptime(s, "%Y-%m-%d").date()
    diff = (date_obj - begin).days
    return diff

# If you inspect temperatures.csv, you'll see the two reporting stations have string identifiers - USC00282644
# and USC00281335.  We will end up converting these into integer ids - and the simplest way is to just
# use the rightmost digit - which is unique.
def station_to_int(s):
    return int(str(s)[-2:-1])

# Loading the CSV into `numpy`
Replace the contents of this cell so that `data` is a new `numpy` array representing the data found in `temperatures.csv`.  This step is critical - you **must** load the data according to the following specifications:

- Utilize `np.genfromtxt` to read `temperatures.csv`.  Consult the documentation for `genfromtext` online in order to do so with the following specifications.
- Skip the header - the data file contains headers, by for `numpy`, we want our data to be all numbers.
- The `dtype` of the array should be `int`.
- The data file has 7 columns, but we only want the following - station, date, tmax, and tmin.  Make sure you only include those columns - see the `usecols` parameter for `genfromtxt` for this.
- The delimiter should be set to a comma `,`.
- Two of the columns are not integers, which if left as-is, would result in `numpy` loading this data as a **structured array**.  This means instead of having a two dimensional array of integers (which we want), we'd get a one dimensional array of tuples!  Use the `converters` parameter to supply two converter functions: one for station (`station_to_int` as defined in the above cell) and for the date (`date_to_day` as defined in above as well).

The result (go ahead and print `data.shape`) should be **a two dimensional integer array** with 60,084 rows and 4 columns.

In [9]:
data = # Use genfromtxt to load temperatures.csv as described above.

data.shape # Should print as (600084, 4)


(60084, 4)

# Collapsing Station Samples
The two reporting stations in the data set report overlapping time periods.  USC00282644 starts in 1893 and stops in the 1970's.  The other station picks up in 1931 and runs until 2020.  Our first step in *reshaping* the data is to convert to a `numpy` array where each individual date is represented once, and for dates where both stations have entries, average the values for tmax and tmin.  This is a reasonable thing to do - since the reporting stations are both within roughly the same geographical region.

## Step 1 - find the unique dates
Use the array splicing operation ([]) to obtain only the date column (should be column 1 in your array).  The resulting shape of the array should be (60,084, 1).  Next, use `numpy`'s  `unique` function to obtain the a non-repeating list of dates, and **store this in a variable called `dates`**.

## Step 2 - use list comprehension to get the mean temperatures
Recall (or look up!) the mechanics of list comprehension.  From the `dates` array, we can create a list comprehension to return a python list of arrays - where each array contains the date, and the mean temperature values.  We'll be doing this a lot in this analysis, so I've provided this step in the cell to do this so you can use it as a reference later.

## Step 3 - construct numpy array for daily temperatures
The result of Step 2 is a python list of lists... which won't be fast to work with later.  Construct a **new `numpy` array, of `dtype` `int` out of the `m` variable.** Name this new array `daily` - it holds daily temperature data.

In [10]:
# Step 1
# You need to do Step 1 as described above, make sure you store the unique 
# list of dates in a variable named `dates`
dates = # the unique list of dates

dates.shape # This should print out (45809,), there are 45,809 dates represented

(45809,)

In [11]:
# Step 2 is provided for you, you'll need to do this type of work on your own later...
m = [
    [   # Note that for each day, we are building an array (normal Python array)
        d,  # first element is the day (which is an integer)
        
        # second element is the average of tmax among the entries for the given day
        # data[data[:, 1] == d] is selecting rows from data (there will be 1 or 2)
        # where the row's column 1 (the date) matches the given date (d).  This is
        # a critical technique - we are GROUPING BY DAY.  Once we have those rows, 
        # we use numpy's mean to compute the average, round it, and convert it to 
        # an integer.
        int(round(data[data[:, 1] == d][:, 2].mean())), 
                        
        # third element is similar to the second, but for tmin.
        int(round(data[data[:, 1] == d][:, 3].mean())),
    ]
    for d in dates  # The source of the comprehension is the numpy array of unique dates
]

# When you run this cell - notice how long it takes (it's slow!).  This is primarily because
# we are doing this in a pure Python list comprehension, numpy can't do what we are doing within
# the CPython native infrastructure.  Keep this in mind... whenever you can, do things exclusively
# with numpy!

len(m) # This will be the same as the length of the unique dates (45809)

45809

In [12]:
daily = # construct np array from m, with int data type

daily.shape  # This should be (45809, 3)

(45809, 3)

# More reshaping
The daily array is close to what we will need - but to answer the questions we are looking at, we need easy ways to filter data by year, month, and the day of the year.  Often it's best to do this reshaping up front, constructing an array that will make the rest of the analysis easier. 

## Step 1 - Obtain a list of [year, month, day_of_year] lists
For each date (column 0 in `daily`), call `day_transform` within a list comprehension to create a list of lists, where each list contains the year, month, and day of year.

## Step 2 - Append the date lists to the numpy array
Use `numpy`'s `hstack` method to do a column-wise append of `daily` and the list you created in step 1.  Store the result in `tmps`.

In [13]:
days = # use day_transform to get list of records containing year, month, and day of year
tmps = # use hstack to append days to daily to obtain tmps

tmps.shape # This should print (45809, 6)
# Column 0:  Date (integer days from 1/1/1893)
# Column 1:  TMAX
# Column 2:  TMIN
# Column 3:  Year (integer, year component of the date)
# Column 4:  Month (1-12), month component of the date.
# Column 5:  Day of Year (0-365) 

(45809, 6)

# Hottest and Coldest Days
Questions 1 and 2 can be solved using similar approaches.
1. Find the unique days of the year - using `np.unique` on the day of year column of tmps
2. Construct a list comprehension for average_maxes and average_mins (two separate lists).
  - For every day of the year, construct a list with the following elements:
    1. The day of the year (as a float)
    2. The mean max/min temperature, but grouping the rows by day of year, and calculating the mean.
3. Construct an `np.array` from each of the two lists you've created, and sort them using `argsort`.  For max, you should sort in reverse order (remember you can reverse a sequence with the splicing operator).  The first entry in the sorted max represents the hottest day, the first entry in the sorted min array represents the coldest day.

For guidance, take a look at Step 2 in Collapsing Station samples from the earlier cell in this notebook.  You are grouping, and calculating the mean - just on different attributes - and this time constructing a list of floats since we don't want to lose precision.


In [14]:
day_of_years = # unique days of year

average_maxes = # 2d np array of day of year (float) and mean max temp on that day of year.

average_mins = # 2d np array of day of year (float) and mean min temp on that day of year.

sorted_maxes = # sort of average_maxes (reversed)
sorted_mins = # sort of average_mins

In [15]:
sorted_maxes[0, :]    # Should report the 200th day of the year, at 87.7 degrees

array([200.        ,  87.70634921])

In [16]:
sorted_mins[0, :]     # Should report the 35th day of the year, at 18.32 degrees

array([35.        , 18.32539683])

# Hottest Septembers on record
To determine the 20 hottest Septembers on record, we have another exercise in aggregation.  T
- Find all unique years - again using `np.unique`, and store the array of unique years in a variable called `years`.
- Construct an `np.array` from a list of lists.  The list of lists should be a list where each element contains a list whose first element is a year (as a float), and the second element is the mean temperature of all reecords (taken from `tmps`) within the month of September for the given year.  
- Store the new `np` array in a variable called `septembers` and sort by mean temperature in reverse order.

Make sure you remove any year that we don't have records for September for!  This one is on you - but to give you a hint, you'll want to remove the years missing September data before constructing the `septembers` array - otherwise you'll get warnings.

In [17]:
septembers[:20]  # 2005, at 84.26, followed by 1930, 1921, and 1961 at 83 and change, 
                 # followed by 1961, 1841, 1915, 2016... concluding with 2019 as the 20th hottest.

array([[2005.        ,   84.26666667],
       [1930.        ,   83.53333333],
       [1921.        ,   83.33333333],
       [1961.        ,   83.26666667],
       [1941.        ,   82.73333333],
       [2016.        ,   82.46666667],
       [1915.        ,   82.46666667],
       [1931.        ,   82.13333333],
       [1914.        ,   81.73333333],
       [1959.        ,   81.56666667],
       [1895.        ,   81.26666667],
       [1998.        ,   81.23333333],
       [1916.        ,   81.13333333],
       [1925.        ,   81.03333333],
       [1983.        ,   81.        ],
       [2007.        ,   80.93333333],
       [1898.        ,   80.8       ],
       [1968.        ,   80.8       ],
       [2019.        ,   80.63333333],
       [1922.        ,   80.6       ]])

# Heat wave in 2016
The final question we want answers to is when the longest stretch of days in 2016 where the daily temperature elevated over 2 degrees above the average temperature for that day of the year.  To do this, we'll compute the average maximum temperature for each day of the year (0-365), calculate the difference between those averages and each day of 2016, and then find the maximum stretch of days that were +2 degrees over the yearly norm for that day.

We've already calculated the average maximum temperature for each day of the year - these are stored in `average_maxes` from the calculation of the hottest days.  

Our first step is to construct an array containing the max temperature for each day of the year in 2016.  We'll do this just like we calculated the September array above:

- Construct a list comprehension, for each day of the year (`day_of_years` from cells above)
- For each day of the year, construct an array whose first element is the day of the year (as a float) and second element is the max temperature on that day in 2016 from the `tmps` array.
- The result of the list comprehension is a list of lists, and use it to construct a 2D float np.array called `daily_2016`

In [19]:
daily_2016.shape  # (366, 2)

(366, 2)

Next, we can calculate the delta between the average temperature on the given day and the temperature on the same day in 2016.  You can do this by subtracting `average_maxes` from `daily_2016`.  Do so in a way that creates a simple 1D element - containing 366 deltas.


In [20]:
delta.shape # (366,)

(366,)

Now create a filter out of `delta`, filtering for values greater than or equal to 2.0
The mask is a new array of 366 true or false values.

In [21]:
mask # You should see lots of True / False values - representing whether the given day was hotter than average.

array([ True, False,  True,  True, False, False, False,  True,  True,
        True,  True, False,  True, False, False,  True,  True, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False,  True,  True, False, False,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True,  True, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True, False, False,
       False,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False, False,

No comes the hardest part - where is the longest streak of hot days?  There is no magic `np` calculation here - you'll need to walk through the array within a loop.  Recommended strategy:

1. Use a variable (`cur`) to host an arry whose first element is the start day of the current streak, and second element is the count of days since the start that were hot.  This won't have a value in the beginning.
2. Use another variable (`longest`) to hold the longest streak you've found so far.  This will be a similar array as `cur` and begin uninitialized as well.
3. Use the `enumerate` function to iterate over the `mask` array.  
  - If the day is hot (True), then if there is a current active streak, add to it's count (it's second element)
  - If the day is hot (True), but there is no active streak, start it by initializing `cur` to contain the current day and a 1 as the streak length.
  - If the day is not hot (False), if there is an active streak, see if the active streak is longer than the longest streak you've seen - if so, replace longest with cur.
  - If the day is not hot (False), regardless of whether there is an active streak or not, you should conclude the iteration by setting `cur` back to `None`.
4. After you've iterated across the entire year - you should make one more check.  If you've ended the loop with an active streak, check if the active streak is longer than the longest you've seen - and set longest accordingly.  Remember, there is no reason we can't have an unusually warm December!

In [22]:

# If you've done this all correctly, the heat wave of 2016 is day 247-263, 16 days of unusually hot weather.
print("Longest stretch from", longest[0], "to", longest[0] + longest[1])

Longest stretch from  247 to 263


Can you use `timedelta` and date's `strftime` to prove these days represent September 4th through September 20th?

'September 04 - September 20'